In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Construct and print the path
functions_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'functions'))
print("Adding to sys.path:", functions_path)

# Add to sys.path if not already there
if functions_path not in sys.path:
    sys.path.append(functions_path)

# Print files to verify
print("Files in functions folder:", os.listdir(functions_path))

# Now try importing
import prospect_optimizer as po

Adding to sys.path: c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\speciale_repo\functions
Files in functions folder: ['CommonFunctions.py', 'ConditionalAssetPricingCommonFunction.py', 'conditionalAssetPricingLogMarginalLikelihoodTauClass.py', 'conditionalAssetPricingOOSPrediction.py', 'expected_returns_puller.py', 'GammaFunctions.py', 'prospect_optimizer.py', 'tictoc.py', 'writeProfLatexTable.py', '__pycache__']


In [2]:
#Requirements
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
import pandas as pd
import statsmodels.api as sm
from scipy.stats import mannwhitneyu, levene, f_oneway

import os
import sys
import time
import pickle as pkl


# Add the functions folder to the Python path
functions_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'functions'))
print("Files in functions folder:", os.listdir(functions_path))
print(functions_path)
import prospect_optimizer as po

Files in functions folder: ['CommonFunctions.py', 'ConditionalAssetPricingCommonFunction.py', 'conditionalAssetPricingLogMarginalLikelihoodTauClass.py', 'conditionalAssetPricingOOSPrediction.py', 'expected_returns_puller.py', 'GammaFunctions.py', 'prospect_optimizer.py', 'tictoc.py', 'writeProfLatexTable.py', '__pycache__']
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\speciale_repo\functions


In [3]:
homeDir = r'C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\speciale_repo\Complemetary Code Files for Submission\\'
dataDir = homeDir + r'Data\\'

n_predictors_to_use = 2 
assert 1 <= n_predictors_to_use <= 13, "You must choose between 1 and 13 predictors"

factors = pd.read_csv(dataDir + 'factors-20.csv')
factors = factors.drop(columns=['MKTRF','SMB*','MKT','CON','IA', 'ROE', 'ME'])  # remove redundant/duplicated
factorsNames = factors.columns.drop('Date')
for col in factors.columns:
   if col != 'Date':
        factors[col] = factors[col]


# === Step 5: Load and clean predictors ===
predictors = pd.read_csv(dataDir + 'Z - 197706.csv')
if 'Unnamed: 0' in predictors.columns:
    predictors = predictors.drop(columns='Unnamed: 0')

# Define full predictor index list
full_predictor_list = predictors.columns.drop('Date')
predictors[full_predictor_list] = predictors[full_predictor_list] / 100
significantPredictors = np.arange(n_predictors_to_use)
predictorsNames = full_predictor_list[significantPredictors]

# Subset actual predictor data
predictors = predictors[['Date'] + predictorsNames.tolist()]

In [4]:

# === Step 6: Truncate all to shared sample size and index ===
start_date = '1997-01-01'
end_date = '2006-12-01'
date_index = pd.date_range(start=start_date, end=end_date, freq='MS')
n = len(date_index)

#returns = returns.iloc[:n].copy()

#returns_sim = returns_sim.iloc[:n].copy()

factors = factors.iloc[:n].copy()
predictors = predictors.iloc[:n].copy()
# === Convert decimal returns to percent returns ===

#returns.index = date_index
factors.index = date_index
predictors.index = date_index

# === Step 7: Reset index and rename for model compatibility ===
#returns_reset = returns.reset_index().rename(columns={'index': 'Date'})
#returns_reset.drop(columns='Row Labels', inplace=True) 

#returns_sim_reset = returns_sim.reset_index().rename(columns={'index': 'Date'})

factors_reset = factors.reset_index().rename(columns={'index': 'Date'})
predictors_reset = predictors.reset_index().rename(columns={'index': 'Date'})


#returns_reset   = returns.reset_index().rename(columns={'index': 'Date'})
factors_reset   = factors.reset_index().rename(columns={'index': 'Date'})
predictors_reset = predictors.reset_index().rename(columns={'index': 'Date'})

In [5]:
bma_returns = po.calculate_bma_returns(0, factors_reset, predictors_reset, 1.1)

Tau= 1.100000 
Date    2006-03-01 00:00:00
Date                 198608
Name: 110, dtype: object
Date    2006-03-01 00:00:00
Date               8/1/1986
Name: 110, dtype: object
Market SR^2 in the estimation period= 0.021816
ZEstimation mean and std
[-0.0304918 -0.0304101]
[0.00136542 0.00133847]
ZTest mean and std
[-0.03450558 -0.0342835 ]
[0.00075889 0.00072605]
After demeaning
ZEstimation mean and std
[5.34925639e-15 1.79250554e-15]
[1. 1.]
ZTest mean and std
[-2.93959451 -2.89390993]
[0.55579342 0.54244947]
REstimation.shape= (0, 0)
RTest.shape= (0, 0)
FEstimation.shape= (110, 14)
FTest.shape= (9, 14)
ZEstimation.shape= (110, 2)
ZTest.shape= (9, 2)
KMax= 14, MMax= 2, T= 110, keyConditionalAPM=1
***** conditionalAssetPricingLogMarginalLikelihoodTauNumba **** 
Model  0 out of 65536
Model  6553 out of 65536
Model  13106 out of 65536
Model  19659 out of 65536
Model  26212 out of 65536
Model  32765 out of 65536
Model  39318 out of 65536
Model  45871 out of 65536
Model  52424 out of 65536

In [6]:
bma_returns

array([0.00242197, 0.00170188, 0.00178322, 0.00144843, 0.00117556,
       0.00393726, 0.00223874, 0.00413841, 0.00183185, 0.00631638,
       0.00282943, 0.00275522, 0.00287849, 0.00550933])

In [9]:

toc = time.time()

parent_dir = os.getcwd() # speciale_repo

### Input parameters
r_hat = 0.0  # Reference return
lambda_ = 1.5  # Base loss aversion coefficient
strategies = ["conservative","aggressive"]  # You can switch between "aggressive" or "conservative"
#lambda_values = [2.5, 1]
#gamma_values = [0.5, 2]

# strategies = ["conservative","aggressive"]
lambda_values = [0.5, 1, 1.5, 2, 2.5]

gamma_values = [0.12,0.2,0.25,0.35,0.5]

#Indlæser returnsdata
#datapath = os.path.join(parent_dir+'/data/returns_data.csv')

# Short data
datapath = os.path.join(parent_dir+'/data/returns.csv')


returns = pd.read_csv(datapath, index_col='Date')

# factors_result = po.optimize_portfolio(r_s, r_hat, lambda_, strategy)


returns_result = po.optimize_portfolio(bma_returns, r_hat, lambda_, gamma = 0.5)
print("Results are generating")
#results = po.resultgenerator(lambda_values, gamma_values, returns, strategies)
print("Done")



# Create returns_conservative
#returns_conservative = {
#    key: value for key, value in results.items() if "conservative" in key
#}

# Create returns_aggressive
#returns_aggressive = {
#    key: value for key, value in results.items() if "aggressive" in key
#}




tic = time.time()
print(f"Elapsed time: {(tic-toc):.2f}")

Results are generating
Done
Elapsed time: 0.02


In [10]:
returns_result.x

array([ 2.53963517e-15, -2.22044605e-16, -3.19189120e-16, -2.49800181e-16,
       -3.05311332e-16, -3.05311332e-16, -2.63677968e-16, -2.35922393e-16,
       -3.19189120e-16,  1.00000000e+00, -2.63677968e-16, -2.63677968e-16,
       -2.63677968e-16, -2.22044605e-16])

In [ ]:
#Data skal være renset og rigtige datolængde
def calculate_bma_returns(rr , ff, zz, tau):
    index_of_estimation = len(ff)
    n_predictors_to_use = 2
    significantPredictors = np.arange(n_predictors_to_use)
    # === Step 1: Instantiate model ===
    model = Model(rr=pd.DataFrame({'': []}),#,returns_reset,
                ff=ff,
                zz=zz,
                significantPredictors=significantPredictors,
                Tau=tau,
                indexEndOfEstimation=index_of_estimation,
                key_demean_predictors=True)
    # === Step 2: Calculate marginal likelihood ===
    CLMLList = []
    (CLMLU, factorsNames, factorsProbabilityU, predictorsNames, predictorsProbabilityU, \
                        T0IncreasedFraction, T0Max, T0Min, T0Avg, T0_div_T0_plus_TAvg, T_div_T0_plus_TAvg, \
                        CLMLR, factorsProbabilityR, predictorsProbabilityR) = \
                            model.conditionalAssetPricingLogMarginalLikelihoodTauNumba()

    CLMLCombined = np.concatenate((CLMLU, CLMLR), axis=0)
    CMLCombined  = np.exp(CLMLCombined - max(CLMLCombined)); CMLCombined = CMLCombined / np.sum(CMLCombined)


    # === Step 3: Calculate posterior probabilities, Maybe? ===
    factorsProbability = np.sum(CMLCombined[0:len(CLMLU)])*factorsProbabilityU + np.sum(CMLCombined[len(CLMLU): ]) *factorsProbabilityR
    predictorsProbability = np.sum(CMLCombined[0:len(CLMLU)])*predictorsProbabilityU + np.sum(CMLCombined[len(CLMLU): ]) *predictorsProbabilityR
    CLMLList.append({"Tau": tau,     \
                            "LMLU" : CLMLU, "LMLR" :CLMLR,  \
                            "factorsProbability"    : factorsProbability,    \
                            "predictorsProbability" : predictorsProbability, \
                            "T0IncreasedFraction" : T0IncreasedFraction,     \
                            "T0Max" : T0Max, "T0Min" : T0Min, "T0Avg" : T0Avg,   \
                            "T0divT0plusTAvg" : T0_div_T0_plus_TAvg, "TdivT0plusTAvg" : T_div_T0_plus_TAvg, \
                            "MisprisingProb" : np.sum(CMLCombined[0: len(CLMLU)])})
    CLMLU = CLMLList[0]["LMLU"]
    CLMLR = CLMLList[0]["LMLR"]
    CLMLCombined = np.concatenate((CLMLU, CLMLR), axis=0)
    CMLCombined  = np.exp(CLMLCombined - max(CLMLCombined)); CMLCombined = CMLCombined / np.sum(CMLCombined)
    print('probability of mispricing    = %f' %(np.sum(CMLCombined[0: len(CLMLU)])))
    print('probability of no mispricing = %f' %(np.sum(CMLCombined[len(CLMLU):  ])))
    factorsProbability = CLMLList[0]["factorsProbability"]
    predictorsProbability = CLMLList[0]["predictorsProbability"]
    # step 4: Calculate the returns
    nModelsInPrediction = 1000
    (returns_OOS , returns_square_OOS, returns_interactions_OOS, \
                            covariance_matrix_OOS, covariance_matrix_no_ER_OOS, \
                            returns_IN, returns_square_IN, returns_interactions_IN, \
                            covariance_matrix_IN, covariance_matrix_no_ER_IN, \
                            cumulative_probability) = \
                            model.conditionalAssetPricingOOSTauNumba(CMLCombined,nModelsInPrediction)
    return(returns_OOS[0])
